In [22]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

# ADVERTISING

In [11]:
df = pd.read_csv("Advertising.csv")

In [12]:
X = df[["TV", "radio", "newspaper"]].values
y = df["sales"].values

In [13]:
modelo = LinearRegression()
modelo.fit(X, y)

beta0_hat = modelo.intercept_
betas = modelo.coef_
print("β0:", beta0_hat)
print("β1 (TV):", betas[0])
print("β2 (radio):", betas[1])
print("β3 (newspaper):", betas[2])

y_pred = modelo.predict(X)
resid = y - y_pred

n = len(y)
p = X.shape[1] + 1

RSS = np.sum(resid**2)
sigma2 = RSS / (n - p)

X_with_intercept = np.column_stack([np.ones(n), X])

cov_matrix = sigma2 * np.linalg.inv(X_with_intercept.T @ X_with_intercept)

SE_betas = np.sqrt(np.diag(cov_matrix))
print()
print("SE(β0):", SE_betas[0])
print("SE(β1 TV):", SE_betas[1])
print("SE(β2 radio):", SE_betas[2])
print("SE(β3 newspaper):", SE_betas[3])

β0: 2.938889369459412
β1 (TV): 0.0457646454553976
β2 (radio): 0.18853001691820448
β3 (newspaper): -0.0010374930424763285

SE(β0): 0.31190823632179143
SE(β1 TV): 0.0013948968069749735
SE(β2 radio): 0.008611233967301958
SE(β3 newspaper): 0.005871009647086364


# DEFAULT

In [14]:
df=pd.read_csv("Default.csv")
df["default"]=df["default"].astype("category")
df["student"]=df["student"].astype("category")
y=df["default"]== "Yes"
x=df["balance"].values.reshape(-1,1)
rl=LogisticRegression()
rl.fit(x,y)
y_pred=rl.predict(x)
y_prob =rl.predict_proba(x)[:,1]

In [15]:
incertidumbre = y_prob * (1 - y_prob)
V = np.diagflat(incertidumbre)
X = df[["balance"]].values
# Se agrega una columna de 1s
X = np.column_stack((np.ones(len(X)), X))
cov = np.linalg.inv(X.T @ V @ X)
se = np.sqrt(np.diag(cov))
z0=rl.intercept_/se[0]
z1=rl.coef_/se[1]


In [16]:
from scipy.stats import norm
p_value0 = 2 * (1 - norm.cdf(abs(z0)))
p_value1 = 2 * (1 - norm.cdf(abs(z1)))
X_f = (df["student"] == "Yes").astype(int).values.reshape(-1,1)
X1 = np.column_stack([np.ones(len(X_f)), X_f])
rl=LogisticRegression()
rl.fit(X_f,y)
y_pred=rl.predict(X_f)
y_prob =rl.predict_proba(X_f)[:,1]
print(rl.intercept_,rl.coef_)
incertidumbre = y_prob * (1 - y_prob)
V = np.diagflat(incertidumbre)
cov = np.linalg.inv(X1.T @ V @ X1)
se = np.sqrt(np.diag(cov))
z0=rl.intercept_/se[0]
z1=rl.coef_/se[1]
p_value0 = 2 * (1 - norm.cdf(abs(z0)))
p_value1 = 2 * (1 - norm.cdf(abs(z1)))

[-3.50257249] [[0.39620888]]


In [17]:
import pandas as pd

beta0_hat = rl.intercept_[0]
beta1_hat = rl.coef_[0][0]

SE_beta0 = se[0]
SE_beta1 = se[1]

print("Error estándar de β0:", SE_beta0)
print("Error estándar de β1:", SE_beta1)


Error estándar de β0: 0.07066142671069267
Error estándar de β1: 0.11522061447776961


# ADVERTISING CON BOOTSTRAPPING

In [18]:
df = pd.read_csv("Advertising.csv")
X = df[["TV", "radio", "newspaper"]].values
y = df["sales"].values


In [10]:
B = 1000
boot_coefs = []

n = len(X)

for b in range(B):
    idx = np.random.choice(n, n, replace=True)
    X_b = X[idx]
    y_b = y[idx]

    model_boot = LinearRegression()
    model_boot.fit(X_b, y_b)

    boot_coefs.append([model_boot.intercept_, *model_boot.coef_])

boot_coefs = np.array(boot_coefs)

media_boot = np.mean(boot_coefs, axis=0)
std_boot = np.std(boot_coefs, axis=0)

print("Mean of bootstrap coefficients (Intercept, TV, radio, newspaper):", media_boot)
print("Standard deviation of bootstrap coefficients (Intercept, TV, radio, newspaper):", std_boot)

Mean of bootstrap coefficients (Intercept, TV, radio, newspaper): [ 2.95876994e+00  4.56759863e-02  1.88027262e-01 -8.16899828e-04]
Standard deviation of bootstrap coefficients (Intercept, TV, radio, newspaper): [0.33103688 0.00189748 0.01086755 0.00645859]


# DEFAULT BOOTSTRAP

In [20]:
df = pd.read_csv("Default.csv")
df["default"] = df["default"].astype("category")
df["student"] = df["student"].astype("category")

y = (df["default"] == "Yes").values
x = df["balance"].values.reshape(-1, 1)

In [21]:
B = 1000
boot_coefs_logistic = []

n = len(x)

for b in range(B):

    idx = np.random.choice(n, n, replace=True)

    x_b = x[idx]
    y_b = y[idx]


    model_boot_logistic = LogisticRegression()
    model_boot_logistic.fit(x_b, y_b)


    boot_coefs_logistic.append([
        model_boot_logistic.intercept_[0],
        model_boot_logistic.coef_[0][0]
    ])

boot_coefs_logistic = np.array(boot_coefs_logistic)

media_boot_logistic = np.mean(boot_coefs_logistic, axis=0)
std_boot_logistic = np.std(boot_coefs_logistic, axis=0)

print("Mean of bootstrap coefficients (Intercept, balance):", media_boot_logistic)
print("Standard deviation of bootstrap coefficients (Intercept, balance):", std_boot_logistic)


Mean of bootstrap coefficients (Intercept, balance): [-1.06672258e+01  5.50818762e-03]
Standard deviation of bootstrap coefficients (Intercept, balance): [3.68445123e-01 2.23960133e-04]


# PUNTO 3
## Tabla comparativa- adversiting
| Dataset     | Método          | β0 (intercept) | SE(β0) | β1 (TV) | SE(β1) | β2 (radio) | SE(β2) | β3 (newspaper) | SE(β3) |
| ----------- | --------------- | -------------- | ------ | ------- | ------ | ---------- | ------ | -------------- | ------ |
| Advertising | Analítico (OLS) | 2.9389         | 0.3119 | 0.0458  | 0.0014 | 0.1885     | 0.0086 | -0.0010        | 0.0059 |
| Advertising | Bootstrap       | 2.9588         | 0.3310 | 0.0457  | 0.0019 | 0.1880     | 0.0109 | -0.0008        | 0.0065 |

## Tabla comparativa - Default
| Método                       | β₀ (Intercept) | SE(β₀)  | β₁ (Balance) | SE(β₁)   |
| ---------------------------- | -------------- | ------- | ------------ | -------- |
| Analítico  | -10.6754       | 0.36803 | 0.005512     | 0.000224 |
| Bootstrap  | -10.6672       | 0.36845 | 0.005508     | 0.000224 |


Los resultados obtenidos por ambos métodos son extremadamente similares tanto en las estimaciones de las betas como en sus errores estándar. Con esto podemos decir que en ambos casos se están capturando casi la misma estructura de variabilidad en los datos.



#REGULARIZACIÓN L2

In [23]:
np.random.seed(14)

df = pd.read_csv("Advertising.csv")
X = df[["TV", "radio", "newspaper"]].values
y = df["sales"].values

def objective(alpha):
    ridge = make_pipeline(
        StandardScaler(),
        Ridge(alpha=alpha, fit_intercept=True)
    )
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(
        ridge, X, y, cv=cv, scoring="neg_mean_squared_error"
    )
    return scores.mean()

X_alpha = np.random.uniform(0.001, 100, 3).reshape(-1, 1)
y_score = np.array([objective(a[0]) for a in X_alpha]).reshape(-1, 1)

kernel = 1.0 * RBF(length_scale=1.0)
gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=10,
    alpha=1e-6,
    normalize_y=True
)

x_v = np.linspace(0.001, 100, 200).reshape(-1, 1)


for i in range(5):
    gp.fit(X_alpha, y_score)

    y_pred, std = gp.predict(x_v, return_std=True)

    idx_max = np.argmax(y_pred)
    alpha_candidate = x_v[idx_max][0]

    y_real = objective(alpha_candidate)

    X_alpha = np.vstack([X_alpha, [[alpha_candidate]]])
    y_score = np.vstack([y_score, [[y_real]]])

# --- 7. Mejor alpha encontrado ---
best_idx = np.argmax(y_score)
best_alpha = X_alpha[best_idx][0]

print(f"Mejor alpha (L2) encontrado con GP: {best_alpha:.6f}")
print("Scores evaluados:", y_score.ravel())

Mejor alpha (L2) encontrado con GP: 0.001000
Scores evaluados: [-4.33603061 -5.41876208 -5.83315038 -2.96508757 -2.96508757 -2.96508757
 -2.96508757 -2.96508757]


In [24]:
from sklearn.linear_model import Ridge

B = 1000
boot_coefs_ridge = []
n = len(X)

for b in range(B):
    idx = np.random.choice(n, n, replace=True)
    X_b = X[idx]
    y_b = y[idx]

    ridge = Ridge(alpha=best_alpha, fit_intercept=True)
    ridge.fit(X_b, y_b)

    boot_coefs_ridge.append([ridge.intercept_, *ridge.coef_])

boot_coefs_ridge = np.array(boot_coefs_ridge)

media_ridge = np.mean(boot_coefs_ridge, axis=0)
std_ridge = np.std(boot_coefs_ridge, axis=0)

print("Media coeficientes Ridge (Intercept, TV, radio, newspaper):", media_ridge)
print("Desviación estándar Ridge (Intercept, TV, radio, newspaper):", std_ridge)


Media coeficientes Ridge (Intercept, TV, radio, newspaper): [ 2.95034967e+00  4.57726897e-02  1.88160026e-01 -9.97825274e-04]
Desviación estándar Ridge (Intercept, TV, radio, newspaper): [0.33324336 0.00194977 0.01045151 0.00641113]


## Tabla comparativa de advertising
| Método              | β₀      | SE(β₀)  | β₁ (TV) | SE(β₁)  | β₂ (radio) | SE(β₂)  | β₃ (newspaper) | SE(β₃)  |
| ------------------- | ------- | ------- | ------- | ------- | ---------- | ------- | -------------- | ------- |
| **OLS Analítico**   | 2.9389  | 0.3119  | 0.04576 | 0.00139 | 0.18853    | 0.00861 | -0.00104       | 0.00587 |
| **OLS Bootstrap**   | 2.9588  | 0.3310  | 0.04568 | 0.00190 | 0.18803    | 0.01087 | -0.00082       | 0.00646 |
| **Ridge Bootstrap** | 2.95035 | 0.33324 | 0.04577 | 0.00195 | 0.18816    | 0.01045 | -0.000998      | 0.00641 |

Los resultados son prácticamente idénticos, las desviaciones éstandar en bootstrap son mayores por poco. Como en bootstrap no se asume ninguna distribución por eso es que refleja un error estándar ligeramente más grande.